# Importing the needed libraries

In [12]:
from selenium import webdriver
from bs4 import BeautifulSoup
from sqlalchemy import create_engine
import re
import pyodbc
import sqlalchemy as sal
import pandas as pd
import time

# Get the Data Via Web Scrapping

## Create a player class to store all the information

In [13]:
class Player():
    def __init__(self):
        self.link = ''
        self.team_link = ''
        self.name = ''
        self.team = ''
        self.number = ''
        self.position = ''
        self.height = ''
        self.weight = ''
        self.last_attended = ''
        self.country = ''
        self.age = ''
        self.birth_date = ''
        self.experience = ''
        self.draft = ''


## Create a function that gets a list of all the players

In [14]:
driver_path = 'C:\drivers\chrome\chromedriver.exe'
site = 'https://www.nba.com/players'
web = 'https://www.nba.com'
players_list = []

position_dict = {'F': 'Foward', 'G': 'Guard', 'C': 'Center', 'C-F': 'Center-Forward', 'G-F': 'Guard-Forward', 'F-G': 'Foward-Guard', 'F-C': 'Foward-Center'}

def get_players_list():
    
    # Download html page

    driver = webdriver.Chrome(executable_path=driver_path)
    driver.get(site)

    # create a soup object with the downloaded html page
    soup = BeautifulSoup(driver.page_source, 'lxml')
    
    # sind the section of the html that contains a player table with "All" the players name
    
    div = soup.find('div', class_='MockStatsTable_statsTable__2edDg')

    # get the players table
    table = div.find('table', class_='players-list')
    
    body = table.find('tbody')

    # identify each player
    for row in body.find_all('tr'):
        player = Player()

        # Get player link
        p_link = row.find('td')
        a = p_link.find('a') 
        player.link = web + a['href']
        
        # Get team link
        t_link = p_link.find_next_sibling()
        b = t_link.find('a')
        
        number_link = t_link.find_next_sibling()
        position = number_link.find_next_sibling()
        position_text = position.text
        
        try:
            player.team_link = web + b['href']
        except:
            player.team_link = 'NULL'
            player.team = 'NULL'
            player.number = 'NULL'
            player.position = position_dict[position_text]
        
        # add player to players list
        players_list.append(player)
           
    return players_list



## Create a function that updates the properties of each player in a list of players

In [15]:
driver_path = 'C:\drivers\chrome\chromedriver.exe'
site = 'https://www.nba.com/players'

def complete_players_list(players_list):
    
    driver = webdriver.Chrome(executable_path=driver_path)
    count = 1
    
    for player in players_list:
        
        # identify the player link
        p_site = player.link
        print(f'{count} of {len(players_list)} players')
        
        # go the the link and download the html
        driver.get(url=p_site)
        
        # Create a soup object
        soup = BeautifulSoup(driver.page_source, 'lxml')
        
        # Get the section we are interested in to use as a reference
        section = soup.find('section', class_='relative overflow-hidden')
        section2 = soup.find('section', class_='relative text-white lg:px-20 PlayerSummary_statsSectionBG__G3Epx')
        
        # Get team name, position, player
        p_tag = section.find('p', class_='t11 md:t2')
        
        if p_tag:
        
            text = p_tag.text.split('|')

            player.team, player.number, player.position = text
            player.number = player.number[2:]
            player.position = player.position[1:]

        # Get player name
        n_text = section.find('p', class_='PlayerSummary_playerNameText__K7ZXO')
        name = n_text.text
        
        nex = n_text.find_next_sibling()
        player.name = name + ' ' + nex.text
        
        # Get the other attributes
        # height
        
        sec = section2.find('div', class_='flex')

        part = sec.find_all('div', class_='PlayerSummary_playerInfo__1L8sx')
        
        at1 = part[0].find('p', 'PlayerSummary_playerInfoValue__mSfou')
        car = at1.text
        pattern = '[0-9]\.[0-9]+'
        player.height = re.findall(pattern, car)[0]
        
        #weight
        
        
        at2 = part[1].find('p', 'PlayerSummary_playerInfoValue__mSfou')
        car2 = at2.text
        pattern2 = '[0-9]+'
        player.weight = re.findall(pattern2, car2)[1]
        
        # country
        
        
        at3 = part[2].find('p', 'PlayerSummary_playerInfoValue__mSfou')
        player.country = at3.text
        
        # last attended

        at4 = part[3].find('p', 'PlayerSummary_playerInfoValue__mSfou')
        player.last_attended = at4.text
        
        # Age
        
        sec2 = section2.find_all('p', 'PlayerSummary_playerInfoValue__mSfou')
        
        car5 = sec2[4].text
        pattern5 = '[0-9]+'
        player.age = re.findall(pattern5, car5)[0]
        
        # birth_date
        
        player.birth_date = sec2[5].text
        
        # draft

        player.draft = sec2[6].text
        
        # Experience

        car8 = sec2[7].text.split(' ')
        player.experience = car8[0]
        
        print('done!\n')
        count += 1
        
    driver.quit()
    return players_list

In [16]:
completed = complete_players_list(get_players_list())

1 of 50 players
done!

1 of 50 players
done!

1 of 50 players
done!

1 of 50 players
done!

1 of 50 players
done!

1 of 50 players
done!

1 of 50 players
done!

1 of 50 players
done!

1 of 50 players
done!

1 of 50 players
done!

1 of 50 players
done!

1 of 50 players
done!

1 of 50 players
done!

1 of 50 players
done!

1 of 50 players
done!

1 of 50 players
done!

1 of 50 players
done!

1 of 50 players
done!

1 of 50 players
done!

1 of 50 players
done!

1 of 50 players
done!

1 of 50 players
done!

1 of 50 players
done!

1 of 50 players
done!

1 of 50 players
done!

1 of 50 players
done!

1 of 50 players
done!

1 of 50 players
done!

1 of 50 players
done!

1 of 50 players
done!

1 of 50 players
done!

1 of 50 players
done!

1 of 50 players
done!

1 of 50 players
done!

1 of 50 players
done!

1 of 50 players
done!

1 of 50 players
done!

1 of 50 players
done!

1 of 50 players
done!

1 of 50 players
done!

1 of 50 players
done!

1 of 50 players
done!

1 of 50 players
done!

1 of 50 pla

In [75]:
 # go the the link and download the html
    
url = 'https://www.nba.com/player/1629121/jaylen-adams/profile'
driver = webdriver.Chrome(executable_path=driver_path)
driver.get(url)

# Create a soup object
soup = BeautifulSoup(driver.page_source, 'lxml')

# Get the section we are interested in to use as a reference
#section = soup.find('section', class_='relative overflow-hidden')
section2 = soup.find('section', class_='relative text-white lg:px-20 PlayerSummary_statsSectionBG__G3Epx')

sec = section2.find('div', class_='flex')

part = sec.find_all('div', class_='PlayerSummary_playerInfo__1L8sx')
at1 = part[0].find('p', 'PlayerSummary_playerInfoValue__mSfou')
pattern = '[0-9]\.[0-9]+'
car = at1.text
carr = re.findall(pattern, car)[0]


sec2 = section2.find_all('p', 'PlayerSummary_playerInfoValue__mSfou')
        

car8 = sec2[7].text
pattern8 = '[0-9]+'
carr2 = re.findall(pattern8, car8)[0]

# birth_date

carr3 = sec2[5].text


print(carr)
print(carr2)
print(carr3)
driver.quit()


1.83
2
May 4, 1996


In [18]:
# convert scraped data to a pandas dataframe

new = [[self.link,self.team_link,self.name,self.team,self.number,self.position,self.height,self.weight,self.last_attended,self.country,self.age,self.birth_date,self.experience,self.draft] for self in completed]

df = pd.DataFrame(new, )

In [21]:
df.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13
0,https://www.nba.com/player/1630173/precious-ac...,https://www.nba.com/team/1610612748/heat/,Precious Achiuwa,Miami Heat,5,Forward,2.03,102,Memphis,Nigeria,21,"September 19, 1999",Rookie,2020 R1 Pick 20
1,https://www.nba.com/player/1629121/jaylen-adams/,NULL,Jaylen Adams,NULL,NULL,Guard,1.83,102,St. Bonaventure,USA,24,"May 4, 1996",2,Undrafted
2,https://www.nba.com/player/203500/steven-adams/,https://www.nba.com/team/1610612740/pelicans/,Steven Adams,New Orleans Pelicans,12,Center,2.11,120,Pittsburgh,New Zealand,27,"July 20, 1993",7,2013 R1 Pick 12
3,https://www.nba.com/player/1628389/bam-adebayo/,https://www.nba.com/team/1610612748/heat/,Bam Adebayo,Miami Heat,13,Center-Forward,2.06,116,Kentucky,USA,23,"July 18, 1997",3,2017 R1 Pick 14
4,https://www.nba.com/player/200746/lamarcus-ald...,https://www.nba.com/team/1610612759/spurs/,LaMarcus Aldridge,San Antonio Spurs,12,Center-Forward,2.11,113,Texas,USA,35,"July 19, 1985",14,2006 R1 Pick 2


In [22]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50 entries, 0 to 49
Data columns (total 14 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   0       50 non-null     object
 1   1       50 non-null     object
 2   2       50 non-null     object
 3   3       50 non-null     object
 4   4       50 non-null     object
 5   5       50 non-null     object
 6   6       50 non-null     object
 7   7       50 non-null     object
 8   8       50 non-null     object
 9   9       50 non-null     object
 10  10      50 non-null     object
 11  11      50 non-null     object
 12  12      50 non-null     object
 13  13      50 non-null     object
dtypes: object(14)
memory usage: 5.6+ KB


In [23]:
df.shape

(50, 14)